In [1]:
import os
import random
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import os
import albumentations as A


cuda


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
#path
#train
train_image = "data/train/"
train_labels = "data/train_labels"
#test
test_image = "data/test"
test_labels = "data/test_labels"
#val
val_image = "data/val"
val_labels = "data/val_labels"

In [3]:
file_list1 = os.listdir(train_image)
file_list2 = os.listdir(train_labels)
file_list1 = [i.split(".")[0] for i in file_list1]
file_list2 = [i.split(".")[0] for i in file_list2]
print(file_list1 == file_list2)

True


In [4]:
file_list1 = os.listdir(val_image)
file_list2 = os.listdir(val_labels)
file_list1 = [i.split(".")[0] for i in file_list1]
file_list2 = [i.split(".")[0] for i in file_list2]
print(file_list1 == file_list2)

True


In [5]:
from PIL import Image
import os

path = train_image

class_0_count = 0
class_255_count = 0

i = 0
for filename in os.listdir(path):
    i+= 1
    if i == 1000:
        break
    with Image.open(os.path.join(path, filename)) as img:
        class_0_count += img.getchannel(0).getcolors()[0][0]
        class_255_count += img.getchannel(0).getcolors()[1][0]

total_count = class_0_count + class_255_count
class_0_weight = class_0_count / total_count
class_255_weight = class_255_count / total_count

print(f"Class 0 weight: {class_0_weight}")
print(f"Class 255 weight: {class_255_weight}")
weight = [class_0_weight, class_255_weight]

Class 0 weight: 0.5488591497566798
Class 255 weight: 0.4511408502433202


In [6]:
sorted([filename.split(".")[0] for filename in os.listdir(train_image)]) == sorted([filename.split(".")[0] for filename in os.listdir(train_labels)])


True

In [7]:
ds_images = []
ds_lables = []

In [8]:
listdir_train_image = sorted(os.listdir(train_image))
j = 0
for i in listdir_train_image:
    j+=1
    if j % 1000 == 0:
        print(j)
        #break

    ds_images.append(Image.open(os.path.join(train_image, i)).convert("RGB"))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [9]:
listdir_train_labels = sorted(os.listdir(train_labels))
j = 0
for i in listdir_train_labels:
    j+=1
    if j % 1000 == 0:
        print(j)
        #break
    ds_lables.append(Image.open(os.path.join(train_labels, i)).convert("L"))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [10]:
ds_images_val = []
ds_lables_val = []

In [11]:
listdir_val_image = sorted(os.listdir(val_image))
for i in listdir_val_image:
    ds_images_val.append(Image.open(os.path.join(val_image, i)).convert("RGB"))

In [12]:
listdir_val_labels = sorted(os.listdir(val_labels))
for i in listdir_val_labels:
    ds_lables_val.append(Image.open(os.path.join(val_labels, i)).convert("L"))

In [29]:
import albumentations as A

aug_transform = A.Compose([
    A.Downscale(scale_min=0.1, scale_max=0.4, p=1),  # Уменьшение размера изображения
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # Добавление гауссовского шума
    A.MedianBlur(blur_limit=7, p=0.5),  # Применение медианного размытия
    A.GaussianBlur(blur_limit=7, p=0.5),  # Применение размытия по Гауссу
    A.OpticalDistortion(distort_limit=0.3, p=0.5),  # Применение оптического размытия
    A.RandomBrightnessContrast(p=0.5),  # Случайное изменение яркости и контрастности
    A.RandomShadow(shadow_roi=(0, 0, 0.5, 1), p=0.5),  # Добавление случайных теней
    A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),  # Добавление эластичной дисторсии
    A.RandomRain(p=0.5),  # Добавление эффекта дождя
    A.RandomSnow(p=0.5)  # Добавление эффекта снега
])



def transform_and_crop(img, mask, size=256):
    
    img = img.resize((size, size), resample=Image.BICUBIC)
    mask = mask.resize((size, size), resample=Image.NEAREST)
    #img, mask = aug_transform(image=img, mask=mask)

    img = np.array(img, dtype=np.float32)
    mask = np.array(mask, dtype=np.int64)
    img = aug_transform(image=img)['image']
    # Нормализуем изображение
    img = img / 255.0
    img = (img - 0.5) / 0.5

    mask[mask > 0] = 1
    mask[mask == 0] = 0

    img = torch.from_numpy(img)
    mask = torch.from_numpy(mask)
    return img, mask


In [30]:
class custom_dataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        mask = self.masks[idx]
        if self.transform:
            img, mask = self.transform(img, mask)
        img = img.permute(2, 0, 1)
        #print(img.shape)
        return img, mask

In [31]:
train_dataset = custom_dataset(images=ds_images, masks=ds_lables, transform=transform_and_crop)
val_dataset = custom_dataset(images=ds_images_val, masks=ds_lables_val, transform=transform_and_crop)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=10)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=10)   

In [16]:
def iou_score(pred, target):
    pred_i = (pred == 1).float()
    target_i = (target == 1).float()
    intersection = (pred_i * target_i).sum()
    union = pred_i.sum() + target_i.sum() - intersection
    if union == 0:
        iou_i = 1
    else:
        iou_i = intersection / union
    return iou_i


In [17]:
model = smp.DeepLabV3Plus(
    encoder_name="resnet50",
    encoder_weights="imagenet",
    in_channels=3,
    classes=2, 
)


In [18]:
model = nn.DataParallel(model, device_ids = [ 0, 1, 2, 3]).cuda()

In [19]:
weight = torch.tensor(weight).cuda()
criterion = nn.CrossEntropyLoss(weight=weight).cuda()
#criterion = DiceLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)


In [20]:
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    running_iou = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        try:
            inputs = inputs.float().cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.squeeze(1).long())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            pred = torch.argmax(outputs, dim=1)
            iou = iou_score(pred, labels.squeeze(1))
            running_iou += iou
        except Exception as e:
            print(e)
            continue
    epoch_loss = running_loss / len(dataloader)
    epoch_iou = running_iou / len(dataloader)
    return epoch_loss, epoch_iou


In [21]:
def validate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    running_iou = 0.0
    num_batches = len(dataloader)

    with torch.no_grad():
        for inputs, labels in dataloader:
            try:
                inputs = inputs.cuda()
                labels = labels.cuda()
                outputs = model(inputs.float())
                loss = criterion(outputs, labels.squeeze(1).long())
                running_loss += loss.item()
                pred = torch.argmax(outputs, dim=1)
                running_iou += iou_score(pred, labels.squeeze(1))
            except Exception as e:
                print(e)

        epoch_loss = running_loss / num_batches
        epoch_iou = running_iou / num_batches

    return epoch_loss, epoch_iou


In [22]:
import logging
import matplotlib.pyplot as plt

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs):
    best_loss = float('inf')
    train_loss, val_loss, train_iou, val_iou = [], [], [], []
    for epoch in range(num_epochs):
        try:
            epoch_train_loss, epoch_train_iou = train_epoch(model, train_dataloader, criterion, optimizer)
        except Exception as e:
            logger.error(f"Error in training epoch {epoch + 1}: {e}")
            print(e)
        train_loss.append(epoch_train_loss)
        train_iou.append(epoch_train_iou)
        try:
            epoch_val_loss, epoch_val_iou = validate(model, val_dataloader, criterion)
        except Exception as e:
            logger.error(f"Error in validation epoch {epoch + 1}: {e}")
            print(e)
        val_loss.append(epoch_val_loss)
        val_iou.append(epoch_val_iou)
        # Save the best model
        if epoch_val_loss < best_loss:
            best_loss = epoch_val_loss
            torch.save(model.state_dict(), "best_model.pth")
            logger.info("Saved best model")
        torch.save(model.state_dict(), "last_model.pth")

        logger.info("******************************")
        logger.info(f"Epoch {epoch+1}/{num_epochs}")
        logger.info(f"Train Loss: {epoch_train_loss:.4f}, IoU: {epoch_train_iou:.4f}")
        logger.info(f"Validation Loss: {epoch_val_loss:.4f}, IoU: {epoch_val_iou:.4f}")

    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(train_loss, label="Train Loss")
    ax1.plot(val_loss, label="Validation Loss")
    ax1.set(xlabel='Epoch', ylabel='Loss')
    ax1.legend()
    ax2.plot(train_iou, label="Train IoU")
    ax2.plot(val_iou, label="Validation IoU")
    ax2.set(xlabel='Epoch', ylabel='IoU')
    plt.show()


In [ ]:
train_model(model, train_dataloader, val_dataloader, criterion, optimizer, 1000)

INFO:__main__:Saved best model
INFO:__main__:******************************
INFO:__main__:Epoch 1/1000
INFO:__main__:Train Loss: 0.5709, IoU: 0.5752
INFO:__main__:Validation Loss: 0.5446, IoU: 0.6608
INFO:__main__:******************************
INFO:__main__:Epoch 2/1000
INFO:__main__:Train Loss: 0.5632, IoU: 0.5823
INFO:__main__:Validation Loss: 0.5925, IoU: 0.5205
